In [12]:
검색할시도명 = '부산광역시'
검색할시군구명 = '서구'
# 검색할시도명 = '전라북도'
# 검색할시군구명 = '고창군'

# 이거 띄어쓰기 잘해야함 '전라남도 신안군',"",x
import re
def umd(x):
    umd = re.sub('부산광역시 서구',"",x)
    umd = umd.lstrip()
    comp = re.compile('[가-힣]+')
    umd = comp.match(umd)
    umd = umd.group()
    return umd

# import re
# def umd(x):
#     umd = re.sub('전라북도 고창군',"",x)
#     umd = umd.lstrip()
#     comp = re.compile('[가-힣]+')
#     umd = comp.match(umd)
#     umd = umd.group()
#     return umd

u1 = ['동대신1동','동대신2동','동대신3동','서대신1동','서대신3동',
      '서대신4동','부민동','아미동','초장동','충무동','남부민1동',
      '남부민2동','암남동']
# u1 =['고창읍','고수면','아산면','무장면','공음면','상하면','해리면','성송면','대산면','심원면','흥덕면','성내면','신림면','부안면']

In [16]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


data = pd.read_csv('dataset/전국스마트가로등표준데이터.csv',encoding='cp949')

data.isnull().sum()

data.columns

data['시도명'].unique()

data['시도명'] = data['시도명'].replace('14','전라남도')

data[data['시도명']==검색할시도명]['시군구명'].unique()

data1 = data[data['시도명']==검색할시도명]
detail_1 = data1[data1['시군구명']==검색할시군구명]


detail_1['읍/면/동'] = detail_1['소재지지번주소'].apply(lambda x : umd(x))

detail_1['설치유무'] = 1

get_col = ['시도명','시군구명','소재지지번주소','읍/면/동','설치유무']
detail_1 = detail_1[get_col]


u2 = detail_1['읍/면/동'].unique().tolist()
us = pd.Series(u1)

detail = pd.DataFrame(us)
detail['설치유무'] = np.nan
detail['시도명'] = data[data['시도명']==검색할시도명]['시도명'].unique()[0]
detail['시군구명'] = data[data['시군구명']==검색할시군구명]['시군구명'].unique()[0]

def 설치유무(x):
    y = ''
    for i in u2 :
        if x == i :
            y = 1
    return y


detail['설치유무'].update(detail[0].apply(설치유무))
detail['설치유무'] = detail['설치유무'].replace("",0)
detail = detail.rename(columns={0:'읍/면/동'})

detail = detail[['시도명','시군구명','읍/면/동','설치유무']]

detail

TypeError: expected string or bytes-like object

In [3]:
fire = pd.read_csv('dataset/화재발생정보.csv', encoding='cp949',index_col='연번')
fire = fire[(fire['시도']==검색할시도명) & (fire['시군구'] == 검색할시군구명)]
fire['읍면동'].value_counts()
group = fire.groupby('읍면동').count()
group = group.reset_index(drop = False)
group['화재발생횟수'] = group['사망']
fire = group[['읍면동','화재발생횟수']]
fire = fire.rename(columns={'읍면동':'읍/면/동'})
fire['화재발생비율'] = fire['화재발생횟수']/fire['화재발생횟수'].sum()
fire['화재발생비율'] = fire['화재발생비율'].round(2)
df = pd.merge(detail,fire,on='읍/면/동',how='outer')
df

,시도명,시군구명,읍/면/동,설치유무,화재발생횟수,화재발생비율
0,전라북도,고창군,고창읍,1.0,24,0.17
1,전라북도,고창군,고수면,1.0,9,0.06
2,전라북도,고창군,아산면,1.0,10,0.07
3,전라북도,고창군,무장면,1.0,13,0.09
4,전라북도,고창군,공음면,1.0,8,0.06
5,전라북도,고창군,상하면,1.0,9,0.06
6,전라북도,고창군,해리면,1.0,8,0.06
7,전라북도,고창군,성송면,1.0,8,0.06
8,전라북도,고창군,대산면,1.0,12,0.09
9,전라북도,고창군,심원면,1.0,4,0.03
